In [15]:
import astropy.io.fits as pf
import numpy as np
import matplotlib.pyplot as plt
import torch
from skimage.transform import rotate

In [16]:
data = pf.open("Combined_DES_images_catalog.fits")

In [17]:
image = data[0].data

In [18]:
for i in range(0,len(image)):
    small = image[i]
    search = np.where(small<=0)
    search_2 = np.where(small>0)
    small[search] = np.min(small[search_2])
    image[i]=small

flat_image = np.log10(image.reshape(2862,2500))
origin_image = image.reshape(2862,2500)

In [19]:
labels = np.copy(data[1].data['spiral'])

In [20]:
image_labels_all = np.zeros((len(labels),2))
for i in range(0,len(image_labels_all)):
    image_labels_all[i,labels[i]]=1

In [21]:
two_D_image_train =np.zeros((len(data[0].data),1,50,50))
for i in range(0,len(data[0].data)):
    two_D_image_train[i,0,:,:]=flat_image[i].reshape(50,50)

#two_D_image_train = torch.from_numpy(np.array(two_D_image_train,dtype=np.float32))

image_train = torch.from_numpy(np.array(two_D_image_train[0:2300],dtype=np.float32))
image_labels = torch.from_numpy(np.array(image_labels_all[0:2300,:],dtype=np.float32))

image_test = torch.from_numpy(np.array(two_D_image_train[2300:,:],dtype=np.float32))
test_labels = torch.from_numpy(np.array(image_labels_all[2300:],dtype=np.float32))

In [53]:
import torch
import torch.nn.functional as F
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        ### 
        ### input 1x50x50 
        self.cnn1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=6,kernel_size=5,padding=0,stride=1), #6x46x46
            torch.nn.BatchNorm2d(6),
            torch.nn.MaxPool2d(kernel_size=2), #6x23x23
            torch.nn.ReLU(),
            #
            )
        self.cnn2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=6, out_channels=6,kernel_size=3,padding=0,stride=1), #6x21x21
            torch.nn.BatchNorm2d(6),
            torch.nn.MaxPool2d(kernel_size=2), #6x10x10
            torch.nn.ReLU(),
            #
            )   
        self.cnn3 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=6, out_channels=6,kernel_size=3,padding=0,stride=1), #6x8x8
            torch.nn.BatchNorm2d(6),
            torch.nn.MaxPool2d(kernel_size=2), #6x4x4
            torch.nn.ReLU(),
        
            )
        
        
        self.fc1 = torch.nn.Sequential(
            torch.nn.Linear(144,2)
        
        )
        
            
       
        #self.fc2 = torch.nn.Linear(10, 10)
    def forward(self, x):
        x = self.cnn1(x)
        x = self.cnn2(x)
        x = self.cnn3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)

        out = x
        return out

In [54]:
model = CNN()

In [55]:
learningRate = 0.01
epochs = 100
criterion = torch.nn.CrossEntropyLoss()
# Just the loss function : here we use the default CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

In [56]:

accuracy_array = []
accuracy_test_array = []
loss_array = []
loss_test_array = []
epoch_array = []
N_total_train = 2300
batch_size=64


for epoch in range(epochs):
    model.train()
    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    for start_index_batch in range(0,N_total_train,batch_size):
        # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
        optimizer.zero_grad()
        end_index = min(start_index_batch + batch_size, N_total_train)
        # get output from the model, given the inputs
        #print(len(color_train[start_index_batch:end_index,:]))
        outputs = model(image_train[start_index_batch:end_index])
        
        # get loss for the predicted output
        loss = criterion(outputs, image_labels[start_index_batch:end_index,:])
        # get gradients w.r.t to parameters
        loss.backward()
        # update parameters
        optimizer.step()
        
    model.eval()
    with torch.no_grad():   
        outputs_all = model(image_train)
        pred_y = torch.max(outputs_all, 1)[1].data.squeeze()
        accuracy = torch.sum((pred_y == torch.max(image_labels, 1)[1].data.squeeze()) / pred_y.size(0))
        #if epoch % 50 ==0:
        #    print(epoch,accuracy,loss)

        epoch_array.append(epoch)

        loss_array.append(float(loss.detach().numpy()))
        accuracy_array.append(float(accuracy.numpy()))


        outputs_test = model(image_test) 
        loss_test = criterion(outputs_test, test_labels)

        pred_y_test = torch.max(outputs_test, 1)[1].data.squeeze()
        accuracy_test = torch.sum((pred_y_test == torch.max(test_labels, 1)[1].data.squeeze()) / pred_y_test.size(0))
        accuracy_test_array.append(float(accuracy_test.numpy()))
        loss_test_array.append(float(loss_test.detach().numpy()))
    
    if epoch % 1 ==0:
        print(epoch,accuracy.numpy(),loss.detach().numpy(), accuracy_test.numpy(),loss_test.detach().numpy())

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x96 and 144x2)

In [ ]:
plt.scatter(epoch_array,accuracy_array)
plt.scatter(epoch_array,accuracy_test_array)
plt.ylim(0.7,1)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')